In [1]:
import sys
sys.path.append('/data/tillson/llm-project/SimPO/scripts')


import logging
import random
import sys

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import logging
import gc


import torch
import transformers
from transformers import AutoModelForCausalLM, set_seed

from alignment import (
    DataArguments,
    # DPOConfig,
    # H4ArgumentParser,
    ModelArguments,
    get_checkpoint,
    get_datasets,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    get_tokenizer,
    # is_adapter_model,
)
from alignment.data import maybe_insert_system_message, is_openai_format
from peft import PeftConfig, PeftModel
from simpo_trainer import SimPOTrainer

import glob


from simpo_config import SimPOConfig
from dataclasses import dataclass, field
from typing import Optional, Literal


import seaborn as sns
import matplotlib.pyplot as plt


/data/tillson/conda-envs/simpo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-09 06:47:41,625] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/tillson/conda-envs/simpo/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/tillson/conda-envs/simpo/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
logger = logging.getLogger(__name__)


In [3]:
from accelerate import Accelerator
from transformers import HfArgumentParser
from scripts import simpo_config
parser = HfArgumentParser((ModelArguments, DataArguments, SimPOConfig))
model_args, data_args, training_args = parser.parse_yaml_file(yaml_file="training_configs/mistral-7b-base-simpo.yaml")


/data/tillson/conda-envs/simpo/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
raw_datasets = get_datasets(
    data_args,
    splits=data_args.dataset_splits,
    configs=data_args.dataset_configs,
    columns_to_keep=["messages", "chosen", "rejected", "prompt", "completion", "label"],
    # seed=training_args.seed,
)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'messages'],
        num_rows: 61135
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'messages'],
        num_rows: 2000
    })
})

In [5]:
column_names = list(raw_datasets["train"].features)

In [6]:
logger = logging.getLogger(__name__)

MISTRAL_CHAT_TEMPLATE = "{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'].strip() + '\n\n' %}{% else %}{% set loop_messages = messages %}{% set system_message = '' %}{% endif %}{% for message in loop_messages %}{% if loop.index0 == 0 %}{% set content = system_message + message['content'] %}{% else %}{% set content = message['content'] %}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + content.strip() + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ ' '  + content.strip() + ' ' + eos_token }}{% endif %}{% endfor %}"

def apply_chat_template(
    example,
    tokenizer,
    task: Literal["sft", "generation", "rm", "simpo"],
    auto_insert_empty_system_msg: bool = True,
    change_template = None,
):
    if change_template == "mistral":
        tokenizer.chat_template = MISTRAL_CHAT_TEMPLATE
    if task in ["sft", "generation"]:
        messages = example["messages"]
        # We add an empty system message if there is none
        if auto_insert_empty_system_msg:
            maybe_insert_system_message(messages, tokenizer)
        example["text"] = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True if task == "generation" else False,
        )
    elif task == "rm":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            chosen_messages = example["chosen"]
            rejected_messages = example["rejected"]
            # We add an empty system message if there is none
            if auto_insert_empty_system_msg:
                maybe_insert_system_message(chosen_messages, tokenizer)
                maybe_insert_system_message(rejected_messages, tokenizer)

            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `rm` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    elif task == "simpo":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            if not is_openai_format(example["chosen"]) or not is_openai_format(example["rejected"]):
                raise ValueError(
                    f"Could not format example as dialogue for `{task}` task! Require OpenAI format for all messages"
                )

            # For DPO/ORPO, the inputs are triples of (prompt, chosen, rejected), where `chosen` and `rejected` are the final turn of a dialogue
            # We therefore need to extract the N-1 turns to form the prompt
            if "prompt" in example and is_openai_format(example["prompt"]):
                prompt_messages = example["prompt"]
                chosen_messages = example["chosen"]
                rejected_messages = example["rejected"]
            else:
                prompt_messages = example["chosen"][:-1]
                # Now we extract the final turn to define chosen/rejected responses
                chosen_messages = example["chosen"][-1:]
                rejected_messages = example["rejected"][-1:]

            # Prepend a system message if the first message is not a system message
            if auto_insert_empty_system_msg:
                maybe_insert_system_message(prompt_messages, tokenizer)

            example["text_prompt"] = tokenizer.apply_chat_template(prompt_messages, tokenize=False)
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            if example["text_chosen"].startswith(tokenizer.bos_token):
                example["text_chosen"] = example["text_chosen"][len(tokenizer.bos_token):]
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
            if example["text_rejected"].startswith(tokenizer.bos_token):
                example["text_rejected"] = example["text_rejected"][len(tokenizer.bos_token):]
        else:
            raise ValueError(
                f"Could not format example as dialogue for `{task}` task! Require either the "
                f"`[chosen, rejected]` or `[prompt, chosen, rejected]` keys but found {list(example.keys())}"
            )
    else:
        raise ValueError(
            f"Task {task} not supported, please ensure that the provided task is one of ['sft', 'generation', 'rm', 'dpo', 'orpo']"
        )
    return example


In [7]:
#####################################
# Load tokenizer and process datasets
#####################################
data_args.truncation_side = "left"  # Truncate from left to ensure we don't lose labels in final turn
tokenizer = get_tokenizer(model_args, data_args)

if "mistral" in model_args.model_name_or_path.lower():
    change_template = "mistral"
else:
    change_template = None
#####################
# Apply chat template
#####################
raw_datasets = raw_datasets.map(
    apply_chat_template,
    fn_kwargs={
        "tokenizer": tokenizer,
        "task": "simpo",
        "auto_insert_empty_system_msg": data_args.auto_insert_empty_system_msg,
        "change_template": change_template,
    },
    num_proc=data_args.preprocessing_num_workers,
    remove_columns=column_names,
    desc="Formatting comparisons with prompt template",
)

# Replace column names with what TRL needs, text_chosen -> chosen and text_rejected -> rejected
for split in ["train", "test"]:
    raw_datasets[split] = raw_datasets[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )

# Log a few random samples from the training set:
for index in random.sample(range(len(raw_datasets["train"])), 3):
    logger.info(f"Prompt sample {index} of the raw training set:\n\n{raw_datasets['train'][index]['prompt']}")
    logger.info(f"Chosen sample {index} of the raw training set:\n\n{raw_datasets['train'][index]['chosen']}")
    logger.info(f"Rejected sample {index} of the raw training set:\n\n{raw_datasets['train'][index]['rejected']}")

torch_dtype = (
    model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
)
quantization_config = get_quantization_config(model_args)

model_kwargs = dict(
    revision=model_args.model_revision,
    trust_remote_code=model_args.trust_remote_code,
    torch_dtype=torch_dtype,
    use_cache=False if training_args.gradient_checkpointing else True,
    device_map=get_kbit_device_map() if quantization_config is not None else None,
    quantization_config=quantization_config,
    attn_implementation=model_args.attn_implementation,
)

In [8]:
model = model_args.model_name_or_path
# seems to require internet

training_args.model_init_kwargs = model_kwargs

checkpoint_dir = "./outputs/mistral-7b-base-simpo-0.25-sample"
checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint-*")
checkpoints.sort()

# Loop through the checkpoint files
for checkpoint in checkpoints:
    print(f"Processing checkpoint: {checkpoint}")
    
    model_kwargs = dict(
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
        torch_dtype=torch_dtype,
        use_cache=False if training_args.gradient_checkpointing else True,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
        attn_implementation=model_args.attn_implementation,
    )
    training_args.model_init_kwargs = None # model_kwargs

    model = AutoModelForCausalLM.from_pretrained(checkpoint)

    # print(model)
    trainer = SimPOTrainer(
        model=model,
        args=training_args,
        train_dataset=raw_datasets["train"],
        eval_dataset=raw_datasets["test"],
        tokenizer=tokenizer,
        peft_config=get_peft_config(model_args),
    )

    del model
    del trainer
    gc.collect()
    torch.cuda.empty_cache()

    # metrics = trainer.predict(raw_datasets["test"])
    # metrics["eval_samples"] = len(raw_datasets["test"])
    # trainer.log_metrics("eval", metrics)
    # trainer.save_metrics("eval", metrics)

logger.info("*** Evaluation complete ***")


##########
# Evaluate
##########
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    

logger.info("*** Training complete! ***")


Processing checkpoint: ./outputs/mistral-7b-base-simpo-0.25-sample/checkpoint-10


Loading checkpoint shards:  33%|███▎      | 1/3 [00:27<00:55, 27.59s/it]


KeyboardInterrupt: 

In [6]:
checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint-*")
training_args_sample = 0.1  # Replace with your actual `training_args.sample` value

# Extract the numerical values from checkpoints and sort
checkpoints = sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))

# Get the maximum checkpoint number
max_checkpoint = int(checkpoints[-1].split('-')[-1])

# Calculate the target value
target = training_args_sample * max_checkpoint

# Find the closest checkpoint without exceeding the target
closest_checkpoint = None
for checkpoint in checkpoints:
    checkpoint_num = int(checkpoint.split('-')[-1])
    if checkpoint_num <= target:
        closest_checkpoint = checkpoint
    else:
        break


In [7]:
closest_checkpoint

'./outputs/mistral-7b-base-simpo-0.25-sample/checkpoint-40'

In [79]:
raw_datasets["test"]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2000
})